In [1]:
# Package Imports and CSV Reads

import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import time
import tqdm

import warnings
warnings.filterwarnings("ignore")

only_23_all = pd.read_csv('player_lists/only_23_all.csv')
only_23_gk = pd.read_csv('player_lists/only_23_gk.csv')
pre_23_all = pd.read_csv('player_lists/pre_23_all.csv')
pre_23_gk = pd.read_csv('player_lists/pre_23_gk.csv')



from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#This will get you player name - need to use in scraper function

test_player = 'https://fbref.com/en/players/f07be45a/matchlogs/2018/summary/Wayne-Rooney-Match-Logs'
response = requests.get(test_player)
soup = BeautifulSoup(response.text,'html.parser')
test_div = soup.find('div', id='meta')
test_div.find('span').text

'Wayne Rooney'

In [3]:
cats = ['summary','passing','passing_types','gca','defense','possession','misc']

goalkeeping_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Shots on Target Allowed','Goals Allowed','Saves','Save %','Clean Sheet','Post Shot xG','PK attempts','Penalty Kicks Allowed','Penalty Kicks Saved','Penalty Kicks Missed','Launched Passes Completed','Launched Passes Attempted','Launched Passes Completion %','Passes Attempted','Passes thrown','% of Passes Launched','Average Pass Length','Goal Kicks Attempted','Goal Kick Avg Length','Goal Kick Launch %','Opponent Crosses Faced','Opponent Crosses Stopped','% of Opponent Crosses Stopped','Defensive Actions outside box','Average Distance From Goal - All Defensive Actions','Match Report']]

summary_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Gls','Ast','PK Made','PKatt','Sh','SoT','CrdY','CrdR','Touches','Tkl','Int','Blocks','xG','npxG','xAG','SCA','GCA','Passes Completed','Passes Attempted','Passing Cmp%','Progressive Passes','Carries','Progressive Carries','Take on Att','Take on Success','Match Report']]

passing_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Total Passes Completed','Total Passes Attempted','Total Passing Completion %','Total Passing Distance','Total Progressive Passing Distance','Short Passes Completed','Short Passes Attempted','Short Passing Cmp%','Medium Passes Completed','Medium Passes Attempted','Medium Passes Completion %','Long Passes Complete','Long Passes Attempted','Long Pass Completion %','Assists','xAG','expected assists','Key Passes','Passes into final third','Passes into penalty area','Crosses into penalty','Progressive Passes','Match Report']]

pass_type_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Passes Attempted','Live Ball Passes','Dead-ball passes','Passes from FK','Through Balls','Switch Passes','Crosses','Throw-ins','Corner Kicks','In-swinging Corner','Outswinging Corner','Straight Corner','Passes Complete','Passes Offside','Passes Blocked','Match Report']]

gca_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','SCA','SCA from Live Pass','SCA from deadball pass','SCA from Take on','SCA from Shot','SCA from foul drawn','SCA from defense action','GCA','GCA from Live Pass','GCA from deadball pass','GCA from Take on','GCA from Shot','GCA from foul drawn','GCA from defense action','Match Report']]

defense_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Tackles','Tackles Won','Tackles in defensive third','Tackles in Mid third','Tackles in Attacking third','Dribblers Tackled','Attempted Tackles on Dribbler','% of succesfull tackles on dribbler','Unsuccesful tackles on dribbler','Blocks','Shots Blocked','Passes Blocked','Interceptions','Tackles + Interceptions','Clearances','Errors','Match Report']]

poss_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Touches','Touches in Def Pen Area','Touches in Def Third','Touches in Mid Third','Touches in Att Third','Touches in Attacking Penalty Box','Live Ball Touches','Attempted Take On','Succesful Take On','Take on Success Rate','Times Tackled During Take On','Tackled During Take on %','Carries','Total Carry Distance','Prg Carry Distance','Carries into final third','Carries into penalty box','Miscontrols','Times Dispossesed','Passes Received','Progressive Passes Received','Match Report']]

misc_columns = [['Date','Day','Comp','Round','Venue','Result','Squad','Opponent','Start','Pos','Min','Yellow Cards','Red Cards','2nd Yellow Card','Fouls','Fouls Drawn','Offsides','Crosses','Interceptions','Tackles Won','Penalty Kicks Won','Penalty Kicks Conceded','Own Goals','Ball Recoveries','Aerials Won','Aerieals Lost','Aerial Duel win %','Match Report']]

gk_df = pd.DataFrame(columns=goalkeeping_columns, dtype=object)
summary_df = pd.DataFrame(columns=summary_columns, dtype=object)
passing_df = pd.DataFrame(columns=passing_columns, dtype=object)
pass_type_df = pd.DataFrame(columns=pass_type_columns, dtype=object)
gca_df = pd.DataFrame(columns=gca_columns, dtype=object)
defense_df = pd.DataFrame(columns=defense_columns, dtype=object)
poss_df = pd.DataFrame(columns=poss_columns, dtype=object)
misc_df = pd.DataFrame(columns=misc_columns, dtype=object)


In [4]:
#Player Scraper Function

def player_scraper(link_list, dataframe, column_list, csv_title):
    i = 0
    for link in tqdm.tqdm(link_list):
        try:
            time.sleep(3)
            
            response = requests.get(link_list[i])
            soup = BeautifulSoup(response.text, 'html.parser')
            name_div = soup.find('div', id='meta')
            player_name = name_div.find('span').text
            i += 1

            data_table = soup.find('table', {'id': 'matchlogs_all'})
            table_body = data_table.find('tbody')
            table_rows = table_body.find_all('tr')

            data = []

            for row in table_rows:
                    cols = row.find_all(['td', 'th'])
                    cols = [col.text.strip() for col in cols]
                    data.append(cols)

            temp_df = pd.DataFrame(data, columns=column_list)
            temp_df['Player_Name'] = player_name
            dataframe = dataframe.append(temp_df, ignore_index=True)
        except Exception as e:
            print(f"Error processing link {link}: {e}")
            continue

    dataframe.to_csv(f"player_data/{csv_title}.csv", index=False)
    return

In [5]:
#Link Creation - Creating list of all links before 2023 and one list of 2023 for both big 5 & MLS.


pre_23_gk_list = []

for squad in range(len(list(pre_23_gk['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{pre_23_gk['Squad_Code'][squad]}/matchlogs/{pre_23_gk['Year'][squad]}/keeper/{pre_23_gk['Text'][squad]}")
        pre_23_gk_list.append(link)

only_23_gk_list = []

for squad in range(len(list(only_23_gk['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{only_23_gk['Squad_Code'][squad]}/matchlogs/{only_23_gk['Year'][squad]}/keeper/{only_23_gk['Text'][squad]}")
        only_23_gk_list.append(link)

pre_23_summary_list = []

for squad in range(len(list(pre_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{pre_23_all['Squad_Code'][squad]}/matchlogs/{pre_23_all['Year'][squad]}/summary/{pre_23_all['Text'][squad]}")
        pre_23_summary_list.append(link)
        
only_23_summary_list = []

for squad in range(len(list(only_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{only_23_all['Squad_Code'][squad]}/matchlogs/{only_23_all['Year'][squad]}/summary/{only_23_all['Text'][squad]}")
        only_23_summary_list.append(link)


pre_23_passing_list = []

for squad in range(len(list(pre_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{pre_23_all['Squad_Code'][squad]}/matchlogs/{pre_23_all['Year'][squad]}/passing/{pre_23_all['Text'][squad]}")
        pre_23_passing_list.append(link)
        
only_23_passing_list = []

for squad in range(len(list(only_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{only_23_all['Squad_Code'][squad]}/matchlogs/{only_23_all['Year'][squad]}/passing/{only_23_all['Text'][squad]}")
        only_23_passing_list.append(link)
        
pre_23_passing_type_list = []

for squad in range(len(list(pre_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{pre_23_all['Squad_Code'][squad]}/matchlogs/{pre_23_all['Year'][squad]}/passing_types/{pre_23_all['Text'][squad]}")
        pre_23_passing_type_list.append(link)
        
only_23_passing_type_list = []

for squad in range(len(list(only_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{only_23_all['Squad_Code'][squad]}/matchlogs/{only_23_all['Year'][squad]}/passing_types/{only_23_all['Text'][squad]}")
        only_23_passing_type_list.append(link)
        
pre_23_gca_list = []

for squad in range(len(list(pre_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{pre_23_all['Squad_Code'][squad]}/matchlogs/{pre_23_all['Year'][squad]}/gca/{pre_23_all['Text'][squad]}")
        pre_23_gca_list.append(link)
        
only_23_gca_list = []

for squad in range(len(list(only_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{only_23_all['Squad_Code'][squad]}/matchlogs/{only_23_all['Year'][squad]}/gca/{only_23_all['Text'][squad]}")
        only_23_gca_list.append(link)

pre_23_defense_list = []

for squad in range(len(list(pre_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{pre_23_all['Squad_Code'][squad]}/matchlogs/{pre_23_all['Year'][squad]}/defense/{pre_23_all['Text'][squad]}")
        pre_23_defense_list.append(link)
        
only_23_defense_list = []

for squad in range(len(list(only_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{only_23_all['Squad_Code'][squad]}/matchlogs/{only_23_all['Year'][squad]}/defense/{only_23_all['Text'][squad]}")
        only_23_defense_list.append(link)
        
pre_23_poss_list = []

for squad in range(len(list(pre_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{pre_23_all['Squad_Code'][squad]}/matchlogs/{pre_23_all['Year'][squad]}/possession/{pre_23_all['Text'][squad]}")
        pre_23_poss_list.append(link)
        
only_23_poss_list = []

for squad in range(len(list(only_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{only_23_all['Squad_Code'][squad]}/matchlogs/{only_23_all['Year'][squad]}/possession/{only_23_all['Text'][squad]}")
        only_23_poss_list.append(link) 
        
pre_23_misc_list = []

for squad in range(len(list(pre_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{pre_23_all['Squad_Code'][squad]}/matchlogs/{pre_23_all['Year'][squad]}/misc/{pre_23_all['Text'][squad]}")
        pre_23_misc_list.append(link)
        
only_23_misc_list = []

for squad in range(len(list(only_23_all['Squad_Code']))):
        link = (f"https://fbref.com/en/players/{only_23_all['Squad_Code'][squad]}/matchlogs/{only_23_all['Year'][squad]}/misc/{only_23_all['Text'][squad]}")
        only_23_misc_list.append(link) 
    

In [ ]:
# All pre 2023 Info! - Get rid of or comment out before setting up auto run.

player_scraper(pre_23_gk_list, gk_df, gk_columns, 'pre_2023_gk')
player_scraper(pre_23_summary_list, summary_df, summary_columns, 'pre_2023_summary')
player_scraper(pre_23_)

In [ ]:
# final test scrape - all 2023 gk data.

player_scraper(test1, passing_df,passing_columns, '2023_passing_test' )
player_scraper(test2, gca_df, gca_columns, '2023_gca_test' )


In [ ]:
list(passing_df.columns)

In [ ]:
test1 = only_23_passing_list[0:5]
test2 = only_23_gca_list[0:5]

In [ ]:
test